In [30]:
# Import Libraries
import pandas as pd
import requests
import pymongo
from bs4 import BeautifulSoup as bs
import requests
from api_keys import weather_api_key
import prettypandas

In [2]:
# Set up Base DF

# create list of team cities and team names
team_name = ['Hawks','Celtics', 'Nets','Hornets','Bulls','Cavaliers','Mavericks','Nuggets','Pistons','Warriors','Rockets','Pacers','Clippers','Lakers','Grizzlies','Heat', 'Bucks','Timberwolves','Pelicans','Knicks','Thunder','Magic','76ers','Suns','Trail Blazers','Kings','Spurs','Raptors','Jazz','Wizards']
team_city_name = ['Atlanta','Boston','Brooklyn','Charlotte','Chicago','Cleveland','Dallas','Denver','Detroit','San Francisco','Houston','Indiana','Los Angeles','Los Angeles','Memphis','Miami','Milwaukee','Minneapolis','New Orleans','New York City','Oklahoma City','Orlando','Philadelphia','Phoenix','Portland','Sacramento','San Antonio','Toronto','Salt Lake City','Washington DC']
team_location = ['Atlanta','Boston','Brooklyn','Charlotte','Chicago','Cleveland','Dallas','Denver','Detroit','Golden State','Houston','Indiana','Los Angeles','Los Angeles','Memphis','Miami','Milwaukee','Minnesota','New Orleans','New York','Oklahoma City','Orlando','Philadelphia','Phoenix','Portland','Sacramento','San Antonio','Toronto','Utah','Washington']
base_df = pd.DataFrame({
                "Team City" : team_location,
                "Team Name" : team_name
})
#base_df = pd.DataFrame(list(zip(team_city, team_name)), columns = ["Team City", "Team Name"])
base_df

,Team City,Team Name
0,Atlanta,Hawks
1,Boston,Celtics
2,Brooklyn,Nets
3,Charlotte,Hornets
4,Chicago,Bulls
5,Cleveland,Cavaliers
6,Dallas,Mavericks
7,Denver,Nuggets
8,Detroit,Pistons
9,Golden State,Warriors


In [3]:
# Set up CSV

file_to_load = "Resources/nba_2017_salary.csv"

salary = pd.read_csv(file_to_load)
salary.head()

,NAME,POSITION,TEAM,SALARY
0,LeBron James,SF,Cleveland Cavaliers,30963450.0
1,Mike Conley,PG,Memphis Grizzlies,26540100.0
2,Al Horford,C,Boston Celtics,26540100.0
3,Dirk Nowitzki,PF,Dallas Mavericks,25000000.0
4,Carmelo Anthony,SF,New York Knicks,24559380.0


In [4]:
# perform groupby and mean to get average salary for each team
team_salary_avg = salary.groupby(["TEAM"]).mean()["SALARY"].rename("Avg. Team Salary")
new_df = pd.DataFrame(team_salary_avg)
new_df.style.format({"Average Salary":"${:,.2f}"})
new_df = new_df.reset_index(drop = False)
new_df

,TEAM,Avg. Team Salary
0,Atlanta Hawks,4.917651e+06
1,Boston Celtics,5.050278e+06
2,Brooklyn Nets,4.435575e+06
3,Charlotte Hornets,5.318635e+06
4,Chicago Bulls,6.620476e+06
5,Cleveland Cavaliers,1.056409e+07
6,Dallas Mavericks,6.697628e+06
7,Denver Nuggets,4.645013e+06
8,Detroit Pistons,6.574762e+06
9,Golden State Warriors,1.118341e+07


In [5]:
# Set up Webscrape

In [6]:
url = "https://www.basketball-reference.com/leagues/NBA_2018_per_minute.html"
page = requests.get(url)
page

<Response [200]>

In [7]:
soup = bs(page.content, 'html.parser')
#print(soup.prettify())

In [8]:
read_table = pd.read_html(url)
#read_table

In [9]:
type(read_table)

list

In [10]:
df = read_table[0]
#df.head(25)

In [11]:
# drop duplicate player names
df = df.drop_duplicates(subset=['Player'], keep='first')
# drop row 20 which is the same as the header
df = df.drop(df.index[20])
# reset index
df = df.reset_index(drop=True)

In [12]:
#df.head

In [13]:
# change age column data type into an integer
df["Age"] = df["Age"].str.replace(',','.').astype('int')

In [14]:
# Create df which only has team and their average age
age_avg = df.groupby(["Tm"]).mean()["Age"].rename("Avg. Team Age")
age_df = pd.DataFrame(age_avg)
age_df = age_df.reset_index(drop=False)
age_df

,Tm,Avg. Team Age
0,ATL,24.944444
1,BOS,25.277778
2,BRK,24.800000
3,CHI,24.133333
4,CHO,25.625000
5,CLE,29.076923
6,DAL,26.631579
7,DEN,25.625000
8,DET,25.642857
9,GSW,28.000000


In [15]:
# drop row with TOT, index 28 and then reset index
age_df = age_df.drop(age_df.index[28])
age_df = age_df.reset_index(drop=True)
age_df

,Tm,Avg. Team Age
0,ATL,24.944444
1,BOS,25.277778
2,BRK,24.800000
3,CHI,24.133333
4,CHO,25.625000
5,CLE,29.076923
6,DAL,26.631579
7,DEN,25.625000
8,DET,25.642857
9,GSW,28.000000


In [16]:
# Combine the base df with the salary df created from the csv file
base_csv = pd.merge(base_df, new_df, left_index=True, right_index=True)
# print base_csv to make sure it aligns
#base_csv
# drop the TEAM column
base_csv = base_csv.drop(columns=['TEAM'])
#base_csv

In [17]:
# Merge the base_csv with age df created from webscrape
team_average = pd.merge(base_csv, age_df, left_index=True, right_index=True)
# print base_csv to make sure it aligns
#team_average 
# drop the Tm column
team_average = team_average.drop(columns=['Tm'])
team_average

,Team City,Team Name,Avg. Team Salary,Avg. Team Age
0,Atlanta,Hawks,4.917651e+06,24.944444
1,Boston,Celtics,5.050278e+06,25.277778
2,Brooklyn,Nets,4.435575e+06,24.800000
3,Charlotte,Hornets,5.318635e+06,24.133333
4,Chicago,Bulls,6.620476e+06,25.625000
5,Cleveland,Cavaliers,1.056409e+07,29.076923
6,Dallas,Mavericks,6.697628e+06,26.631579
7,Denver,Nuggets,4.645013e+06,25.625000
8,Detroit,Pistons,6.574762e+06,25.642857
9,Golden State,Warriors,1.118341e+07,28.000000


In [18]:
# Set up API pull from open weather map
base_url = "http://api.openweathermap.org/data/2.5/weather?&appid=" + weather_api_key + "&q="
#base_url = "http://history.openweathermap.org/data/2.5/history/city?
unit = "imperial"
city_temp = []
city_name = []

In [19]:
for city in team_city_name:
    if city == "Golden State":
        city = "San Francisco"
    else:
        try:
            response = requests.get(base_url + city + "&units=imperial").json()
            #print(response)
            city_name.append(response['name'])
            city_temp.append(response['main']['temp'])
        
        except(KeyError, IndexError):
            print(city + " not found")
        
print("--completed--")
    

--completed--


In [20]:
len(city_name)

30

In [33]:
# add city temperature to the datafram
api_temp = pd.DataFrame({
        "City Name" : city_name,
        "Avg. Temp of City" : city_temp})
api_temp

,City Name,Avg. Temp of City
0,Atlanta,49.55
1,Boston,38.59
2,Brooklyn,44.02
3,Charlotte,44.04
4,Chicago,52.52
5,Cleveland,36.99
6,Dallas,61.30
7,Denver,45.28
8,Detroit,39.97
9,San Francisco,48.96


In [36]:
# merge api with team average df
main_df = pd.merge(team_average, api_temp, left_index=True, right_index=True)
main_df["Avg. Team Age"] = main_df["Avg. Team Age"].round(1)
#main_df["Avg. Team Salary"] = main_df["Avg. Team Salary"].style.format('${0:,.2f}')
main_df["Avg. Team Salary"] = main_df["Avg. Team Salary"].round(2)
main_df["Avg. Temp of City"] = main_df["Avg. Temp of City"].round(2)
main_df = main_df.drop(columns=['City Name'])
#prettypandas(main_df).as_currency(subset='Avg. Team Salary')

In [37]:
main_df

,Team City,Team Name,Avg. Team Salary,Avg. Team Age,Avg. Temp of City
0,Atlanta,Hawks,4917651.17,24.9,49.55
1,Boston,Celtics,5050277.89,25.3,38.59
2,Brooklyn,Nets,4435574.56,24.8,44.02
3,Charlotte,Hornets,5318635.14,24.1,44.04
4,Chicago,Bulls,6620476.00,25.6,52.52
5,Cleveland,Cavaliers,10564086.25,29.1,36.99
6,Dallas,Mavericks,6697628.33,26.6,61.30
7,Denver,Nuggets,4645013.38,25.6,45.28
8,Detroit,Pistons,6574761.50,25.6,39.97
9,Golden State,Warriors,11183408.62,28.0,48.96
